<a href="https://colab.research.google.com/github/Cherrise-exe/longFiles_watsonSpeechToText/blob/main/Speech_To_Text_Long_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install and Import Dependencies

In [56]:
pip install ibm_watson

In [58]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

2. Setup STT Service

In [59]:
apikey = 'u9dE1TYnndD77H6uCTxPazqixP_Yv1Aj6Rz56Ui_tKbu'
url = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/4313f2f0-ec19-4f25-812e-8e12ae0715f2'

In [60]:
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

3. Compress and Split Audio

In [61]:
import subprocess
import os
command = 'ffmpeg -i NASA_Space_Story_1074.wav -vn -ar 44100 -ac 2 -b:a 192k audio.mp3'
subprocess.call(command, shell=True)
command = 'ffmpeg -i audio.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell=True)

0

In [63]:
files = []
for filename in os.listdir('.'):
  if filename.endswith(".mp3") and filename != 'audio.mp3':
    files.append(filename)
files.sort()

In [64]:
files

['000.mp3', '001.mp3', '002.mp3', '003.mp3']

4. Perform Conversion

In [ ]:
results = []
for filename in files:
  with open(filename, 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel', \
                        inactivity_timeout=360).get_result()
    results.append(res)               

In [37]:
text = []
for file in results:
  for result in file['results']:
    text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [38]:
text

[]

In [39]:
with open('output.txt', 'w') as out:
  out.writelines(text)